In [10]:
# Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [11]:
# Load dataset
dataset= pd.read_csv('dataset.csv')
#print(dataset.columns)

# Features to use
features = ['Patient Age', "Mother's age", "Father's age", 'No. of previous abortion',
    'White Blood cell count (thousand per microliter)']
targets = ['Genetic Disorder', 'Disorder Subclass']

# Select only the desired columns
selected_columns = features + targets
dataset = dataset[selected_columns]
'''
# If 'dataset' is a pandas DataFrame
X = dataset[features]
y = dataset[targets]'''

# Replace 'Unknown' with NaN in Target 
dataset['Genetic Disorder'].replace('Unknown', np.nan, inplace=True)
dataset['Disorder Subclass'].replace('Unknown', np.nan, inplace=True)
# Eliminate samples of NaN targets
dataset.dropna(inplace=True)


# Encode categorical variables
label_encoder = LabelEncoder()

# List of categorical columns to encode
categorical_unordered_columns = [
        'Genetic Disorder', 'Disorder Subclass'
]

quantitative_with_unknowns_or_ordered_columns = [
    'Patient Age', "Mother's age", "Father's age", 'No. of previous abortion',
    'White Blood cell count (thousand per microliter)']

dataset_encoded=dataset.copy()

for column in quantitative_with_unknowns_or_ordered_columns:
    dataset_encoded[column] = label_encoder.fit_transform(dataset_encoded[column].astype(str))
dataset=pd.get_dummies(dataset_encoded, columns=categorical_unordered_columns, drop_first=False)


# Alocate features and labels
X = dataset.iloc[:, :-12]  # Features
y = dataset.iloc[:, -12:]  # Labels (last two columns encoded into 3+9)


# Split the data into validation, validation and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75,  random_state=1) # training as 75% 


In [12]:
# Select and train model of classifier

# Best Hyperparameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 10, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10, 'estimator__splitter': 'random'}
# Best Mean Accuracy: 0.2690801625415589

dt = MultiOutputClassifier(tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 1, min_samples_split= 10, splitter = 'random', random_state = 1))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

cv_scores = cross_val_score(dt, X_train, y_train, cv=kf)

print("Cross-validated scores:", cv_scores)
print("Mean cross-validated score:", cv_scores.mean())

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test) 


Cross-validated scores: [0.00701884 0.00554119 0.00738825 0.00738825 0.0088659 ]
Mean cross-validated score: 0.007240487624676764


In [13]:
# Get the 12 targets to evaluate model
y_test_columns = y_test.columns[-12:] 
y_pred_columns = y_pred[:, -12:]

balanced_acc_scores_ = {}

for i, column in enumerate(y_test_columns):
    y_test_ = y_test[column]
    y_pred_ = y_pred_columns[:, i]
    
    balanced_acc_scores_[column] = balanced_accuracy_score(y_test_, y_pred_)



# Use dataset without Unknown values

In [14]:
# Train models without unknown data

# Replace 'Unknown' with NaN
dataset.replace('Unknown', np.nan, inplace=True)

dataset.dropna(inplace=True)


In [15]:
# Alocate features and labels

X = dataset.iloc[:, :-12]  # Features
y = dataset.iloc[:, -12:]  # Labels (last two columns encoded into 3+9)

# split the data into validation, validation and test

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75,  random_state=1) # training as 75%

In [16]:
# Select and train model of classifier

# Best Hyperparameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 10, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10, 'estimator__splitter': 'random'}
# Best Mean Accuracy: 0.2690801625415589

dt = MultiOutputClassifier(tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 1, min_samples_split= 10, splitter = 'random', random_state = 1))

kf = KFold(n_splits=5, shuffle=True, random_state=1)  

cv_scores = cross_val_score(dt, X_train, y_train, cv=kf)

print("Cross-validated scores:", cv_scores)
print("Mean cross-validated score:", cv_scores.mean())

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test) 


Cross-validated scores: [0.00701884 0.00554119 0.00738825 0.00738825 0.0088659 ]
Mean cross-validated score: 0.007240487624676764


In [17]:
# Get 12 targets to evaluate model
y_test_columns = y_test.columns[-12:] 
y_pred_columns = y_pred[:, -12:]

balanced_acc_scores_new = {}

for i, column in enumerate(y_test_columns):
    y_test_new = y_test[column]
    y_pred_new = y_pred_columns[:, i]
    
    balanced_acc_scores_new[column] = balanced_accuracy_score(y_test_new, y_pred_new)


In [18]:
# Metrics and information for each output column
print('\nClassification Report:')
print(classification_report(y_test_, y_pred_))
for column, score in balanced_acc_scores_.items():
    print(f'Balanced Accuracy for {column}: {score}')

print('\n-------------------------REMOVING UNKNOWNS ---------------------------')

print('\nClassification Report:')
print(classification_report(y_test_new, y_pred_new))

for column, score in balanced_acc_scores_new.items():
    print(f'Balanced Accuracy for {column}: {score}')



Classification Report:
              precision    recall  f1-score   support

       False       0.86      0.99      0.92      3874
        True       0.11      0.00      0.01       638

    accuracy                           0.85      4512
   macro avg       0.48      0.50      0.47      4512
weighted avg       0.75      0.85      0.79      4512

Balanced Accuracy for Genetic Disorder_Mitochondrial genetic inheritance disorders: 0.5026591081760271
Balanced Accuracy for Genetic Disorder_Multifactorial genetic inheritance disorders: 0.5011332882539953
Balanced Accuracy for Genetic Disorder_Single-gene inheritance diseases: 0.4997394401038724
Balanced Accuracy for Disorder Subclass_Alzheimer's: 0.5
Balanced Accuracy for Disorder Subclass_Cancer: 0.5
Balanced Accuracy for Disorder Subclass_Cystic fibrosis: 0.5012768959687003
Balanced Accuracy for Disorder Subclass_Diabetes: 0.4996117391800845
Balanced Accuracy for Disorder Subclass_Hemochromatosis: 0.5012299034457719
Balanced Accuracy fo